In [35]:
print("hello world")

hello world


In [36]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nisha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
# Load resume CSV
df = pd.read_csv("../Resume/Resume.csv")
df = df[['Resume_str', 'Category']]
df.head()


,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR


In [38]:
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'<.*?>', ' ', text)      # remove HTML
    text = re.sub(r'[^a-z\s]', ' ', text)   # remove special chars
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text


In [39]:
df['cleaned_resume'] = df['Resume_str'].apply(clean_text)
df.to_csv("../processed_data/cleaned_resumes.csv", index=False)


In [40]:
with open("../job_description/data_analyst.txt", "r") as f:
    job_description = clean_text(f.read())


In [41]:
SKILLS = [
    'python', 'sql', 'excel', 'power bi', 'tableau',
    'machine learning', 'statistics', 'pandas', 'numpy',
    'data analysis', 'data visualization'
]


In [42]:
def extract_skills(text):
    found = []
    for skill in SKILLS:
        if skill in text:
            found.append(skill)
    return found


In [43]:
df['skills'] = df['cleaned_resume'].apply(extract_skills)
df[['Category', 'skills']].head()


,Category,skills
0,HR,"[statistics, data analysis]"
1,HR,[]
2,HR,[excel]
3,HR,[excel]
4,HR,[excel]


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [45]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer("all-MiniLM-L6-v2")

# Use df directly
resume_embeddings = model.encode(
    df["cleaned_resume"].astype(str).tolist(),
    show_progress_bar=True
)

job_embedding = model.encode([job_description])

scores = cosine_similarity(resume_embeddings, job_embedding).flatten()

df["Similarity Score"] = scores

ranked_df = df.sort_values(
    by="Similarity Score",
    ascending=False
)


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 287.24it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 78/78 [01:41<00:00,  1.30s/it]


In [54]:
# ------------------------------------
# Extract skills from job description
# ------------------------------------
job_skills = extract_skills(job_description)

# ------------------------------------
# Find missing skills for each resume
# ------------------------------------
def find_missing_skills(candidate_skills):
    return list(set(job_skills) - set(candidate_skills))

df["missing_skills"] = df["skills"].apply(find_missing_skills)

# ------------------------------------
# Re-rank AFTER adding missing_skills
# ------------------------------------
ranked_df = df.sort_values(
    by="Similarity Score",
    ascending=False
)

# ------------------------------------
# Final ranked output
# ------------------------------------
final_ranked_df = ranked_df[
    ["Category", "Similarity Score", "skills", "missing_skills"]
]

final_ranked_df.head(10)


,Category,Similarity Score,skills,missing_skills
1762,ENGINEERING,0.470981,"[python, sql, excel, tableau, machine learning...","[power bi, numpy]"
1339,AUTOMOBILE,0.461362,"[python, sql, excel, tableau, statistics, pand...","[power bi, machine learning]"
1303,DIGITAL-MEDIA,0.424703,"[python, sql, excel, power bi, tableau]","[numpy, statistics, machine learning, pandas, ..."
1218,CONSULTANT,0.414045,"[python, sql, machine learning, data analysis,...","[excel, numpy, power bi, statistics, pandas]"
1040,SALES,0.395361,"[excel, tableau, machine learning, statistics,...","[numpy, sql, python, power bi, pandas, data vi..."
1340,AUTOMOBILE,0.383040,[excel],"[numpy, sql, python, power bi, statistics, mac..."
662,BUSINESS-DEVELOPMENT,0.372677,"[python, sql, excel, tableau]","[numpy, power bi, statistics, machine learning..."
1530,FINANCE,0.364987,[excel],"[numpy, sql, python, power bi, statistics, mac..."
1142,CONSULTANT,0.364145,"[sql, excel, tableau]","[numpy, python, power bi, statistics, machine ..."
779,HEALTHCARE,0.361590,[excel],"[numpy, sql, python, power bi, statistics, mac..."


In [55]:
final_ranked_df.to_csv(
    "../processed_data/final_ranked_candidates.csv",
    index=False
)

print("✅ Final ranked candidates CSV saved successfully!")


✅ Final ranked candidates CSV saved successfully!
